In [111]:
import pandas as pd
import numpy as np
from pandas import Series
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from zipfile import ZipFile
import requests
from tabulate import tabulate
import datetime
import calendar

In [112]:
%matplotlib inline

# Getting Data

## Zillow Dataset (Zillow API)

### Latest Data

In [113]:
url_zillow = "https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1635114063"
r = requests.get(url_zillow, allow_redirects=True)
open('zillowData.csv', 'wb').write(r.content)

1833609

In [114]:
df_zillow = pd.read_csv('zillowData.csv')
df_zillow.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,...,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31
0,102001,0,United States,Country,NaN,127202.0,127546.0,127908.0,128645.0,129389.0,130130.0,130886.0,131692.0,132515.0,133343.0,134180.0,135035.0,135834.0,136569.0,137316.0,138113.0,138935.0,139741.0,140552.0,141392.0,142224.0,143061.0,143827.0,144559.0,145227.0,145923.0,146658.0,147436.0,148265.0,149137.0,150083.0,151071.0,152048.0,153030.0,153955.0,...,234180.0,235157.0,236133.0,237098.0,238072.0,239064.0,240083.0,241023.0,241832.0,242530.0,243176.0,243777.0,244328.0,244887.0,245520.0,246233.0,246968.0,247795.0,248779.0,249923.0,251208.0,252429.0,253276.0,253982.0,255081.0,256996.0,259505.0,262323.0,265348.0,268391.0,271623.0,274979.0,278635.0,282495.0,287202.0,292730.0,298613.0,304008.0,308632.0,312728.0
1,394913,1,"New York, NY",Msa,NY,224341.0,225681.0,226887.0,229261.0,231472.0,233825.0,236298.0,238858.0,241425.0,243966.0,246407.0,248694.0,250894.0,253316.0,255684.0,258155.0,260499.0,263033.0,265497.0,268163.0,270863.0,273841.0,276584.0,279231.0,281904.0,284508.0,287044.0,289490.0,292077.0,295011.0,298171.0,302075.0,306162.0,310244.0,313865.0,...,468858.0,469559.0,470737.0,472430.0,474272.0,475833.0,476846.0,477722.0,478618.0,479619.0,480304.0,480502.0,480470.0,480198.0,479733.0,479196.0,478902.0,479439.0,480468.0,481943.0,483360.0,484851.0,485660.0,486216.0,487431.0,490018.0,494520.0,500002.0,506026.0,511847.0,516687.0,521426.0,525792.0,530533.0,536044.0,543211.0,551142.0,558738.0,564641.0,569191.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,231193.0,231998.0,233231.0,235576.0,238089.0,240463.0,242893.0,245444.0,247874.0,250096.0,252153.0,254319.0,255984.0,257787.0,259709.0,262388.0,265016.0,267439.0,269916.0,272500.0,275179.0,277835.0,280149.0,282584.0,284591.0,286957.0,289775.0,293359.0,297436.0,301595.0,306829.0,312281.0,317831.0,322787.0,327708.0,...,660952.0,661829.0,662280.0,665537.0,667390.0,666879.0,665719.0,663569.0,662227.0,661796.0,662764.0,663781.0,662846.0,663571.0,666284.0,670585.0,672598.0,674408.0,677384.0,683481.0,690442.0,695566.0,693656.0,690756.0,692497.0,701669.0,712284.0,722248.0,729738.0,736073.0,741209.0,748698.0,756569.0,767615.0,783282.0,803090.0,823992.0,837437.0,846549.0,851308.0
3,394463,3,"Chicago, IL",Msa,IL,170104.0,170505.0,171024.0,172064.0,173166.0,174257.0,175465.0,176820.0,178221.0,179599.0,180850.0,182074.0,183220.0,184233.0,185383.0,186514.0,187907.0,189195.0,190497.0,191815.0,193145.0,194567.0,195747.0,196814.0,197725.0,198786.0,200065.0,201140.0,202344.0,203446.0,204924.0,206433.0,207893.0,209235.0,210196.0,...,236616.0,236915.0,237562.0,238065.0,238839.0,239458.0,239992.0,240733.0,241198.0,241705.0,241764.0,241691.0,241651.0,241582.0,241579.0,241471.0,241479.0,241692.0,242172.0,242780.0,243309.0,243437.0,243269.0,243240.0,244251.0,245915.0,248799.0,251655.0,254456.0,256680.0,258211.0,260534.0,262648.0,265690.0,268442.0,272263.0,276430.0,280547.0,283620.0,286280.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,130035.0,130139.0,130225.0,130437.0,130658.0,130843.0,131042.0,131408.0,131862.0,132391.0,132875.0,133367.0,133579.0,133258.0,132977.0,132859.0,133077.0,133143.0,133215.0,133509.0,133736.0,133938.0,133987.

In [115]:
#remove regiontype
df_zillow.drop(['RegionType'], axis=1,  inplace=True)

### Crosswalk between Zillow and Federally Defined Regions

In [116]:
#https://data.world/zillow-data/crosswalk-between-zillow-and-federally-defined-regions
#url_xtalk = "https://data.world/zillow-data/crosswalk-between-zillow-and-federally-defined-regions/file/CountyCrossWalk_Zillow.csv"
url_xtalk = "http://files.zillowstatic.com/research/public/CountyCrossWalk_Zillow.csv"
rx = requests.get(url_xtalk,allow_redirects=True)
open('zillowXTalk.csv', 'wb').write(rx.content)

229072

In [117]:
df_Xtalk = pd.read_csv('zillowXTalk.csv',sep=',', encoding='gbk')
df_Xtalk.head()

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Hudson,New Jersey,34,17,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1106,394913.0,34017,35620.0
1,Morris,New Jersey,34,27,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1241,394913.0,34027,35620.0
2,Nassau,New York,36,59,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1252,394913.0,36059,35620.0
3,Orange,New York,36,71,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1290,394913.0,36071,35620.0
4,Putnam,New York,36,79,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1343,394913.0,36079,35620.0


In [118]:
df_Xtalk.drop(["StateFIPS", "CountyFIPS", "CBSAName", 'CountyRegionID_Zillow', "StateName", "MetroName_Zillow"], axis=1, inplace=True)

In [119]:
df_Xtalk

,CountyName,MetroRegionID_Zillow,FIPS,CBSACode
0,Hudson,394913.0,34017,35620.0
1,Morris,394913.0,34027,35620.0
2,Nassau,394913.0,36059,35620.0
3,Orange,394913.0,36071,35620.0
4,Putnam,394913.0,36079,35620.0
...,...,...,...,...
3139,Fisher,NaN,48151,NaN
3140,Forest,NaN,42053,NaN
3141,Forest,NaN,55041,NaN
3142,Foster,NaN,38031,NaN


In [120]:
#remove empty row
df_Xtalk.dropna(axis=0, subset=['MetroRegionID_Zillow', 'CBSACode'], inplace=True)
df_Xtalk['CBSACode'] = df_Xtalk['CBSACode'].astype('int')

In [121]:
df_Xtalk.drop_duplicates(subset=['MetroRegionID_Zillow', 'CBSACode'], inplace=True)

In [122]:
#merge zillow data with cross talk data
zillowCBSA = df_zillow.merge(df_Xtalk, left_on='RegionID', right_on='MetroRegionID_Zillow')
#drop MetroRegionID_Zillow

## House Price Index (Fred Economic Data)

In [140]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dfs = []
for state in states:
  url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?mode=fred&id={state}STHPI&cosd=2000-01-01&coed=2021-04-01"
  r = requests.get(url, allow_redirects=True)
  open('data-hpi.csv', 'wb').write(r.content)
  df_state_hpi = pd.read_csv('data-hpi.csv')
  dfs.append(df_state_hpi)

In [141]:
import functools
df_hpi = functools.reduce(lambda left,right: pd.merge(left,right,on='DATE',how='outer'),dfs)
df_hpi

,DATE,ALSTHPI,AKSTHPI,AZSTHPI,ARSTHPI,CASTHPI,COSTHPI,CTSTHPI,DCSTHPI,DESTHPI,FLSTHPI,GASTHPI,HISTHPI,IDSTHPI,ILSTHPI,INSTHPI,IASTHPI,KSSTHPI,KYSTHPI,LASTHPI,MESTHPI,MDSTHPI,MASTHPI,MISTHPI,MNSTHPI,MSSTHPI,MOSTHPI,MTSTHPI,NESTHPI,NVSTHPI,NHSTHPI,NJSTHPI,NMSTHPI,NYSTHPI,NCSTHPI,NDSTHPI,OHSTHPI,OKSTHPI,ORSTHPI,PASTHPI,RISTHPI,SCSTHPI,SDSTHPI,TNSTHPI,TXSTHPI,UTSTHPI,VTSTHPI,VASTHPI,WASTHPI,WVSTHPI,WISTHPI,WYSTHPI
0,2000-01-01,208.29,173.03,200.31,179.76,261.14,259.54,261.66,234.64,261.48,200.84,229.94,230.87,199.82,241.69,206.11,186.03,179.07,218.78,160.11,277.29,238.23,403.94,253.04,225.27,180.97,207.84,209.71,198.49,190.08,253.16,279.56,198.69,335.72,232.85,159.18,216.09,143.93,247.71,234.47,279.11,231.92,197.42,214.47,158.83,236.49,250.44,229.62,269.79,148.06,227.79,150.22
1,2000-04-01,210.06,175.22,202.53,181.48,268.16,265.86,268.67,242.97,266.05,204.42,233.67,230.57,199.74,245.83,207.86,188.53,180.39,220.77,160.72,284.95,240.79,416.85,256.59,231.95,182.26,210.34,211.36,201.35,192.47,261.91,286.28,199.41,345.87,235.80,160.36,218.72,145.65,249.40,237.93,287.61,234.76,201.25,215.66,161.49,236.44,254.39,234.34,272.56,149.11,228.96,151.76
2,2000-07-01,211.71,176.35,205.25,182.57,277.16,272.85,274.52,250.77,271.23,208.70,237.50,233.57,201.38,249.86,210.69,191.03,182.98,222.66,162.62,291.35,245.23,432.13,261.13,237.71,185.03,213.74,214.67,203.16,194.24,271.61,294.71,200.68,355.94,238.83,162.00,221.36,147.97,251.77,240.41,296.69,237.56,204.72,217.84,163.89,238.96,259.63,238.52,276.23,150.41,232.33,153.27
3,2000-10-01,214.38,176.80,208.36,185.01,285.73,278.97,278.33,255.64,274.74,213.09,241.36,239.16,203.56,252.72,212.58,192.45,184.82,225.50,164.00,297.20,249.06,442.58,264.89,242.34,186.34,216.37,217.68,204.63,197.08,279.09,299.15,201.83,361.41,241.44,163.28,224.24,148.95,254.46,243.24,302.23,240.80,204.93,219.79,165.79,242.33,264.54,242.54,279.75,151.74,235.16,156.62
4,2001-01-01,220.65,179.97,213.01,189.61,296.77,286.80,285.16,266.21,280.63,219.23,248.14,246.90,208.82,256.03,217.08,196.53,188.47,229.87,168.27,305.67,254.78,454.48,269.03,248.72,191.09,219.99,221.92,207.28,201.45,285.38,305.94,206.64,369.56,246.68,167.49,228.50,152.40,261.38,249.52,308.28,247.07,209.29,225.25,170.48,247.72,268.88,248.66,286.57,156.06,238.74,157.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2020-04-01,346.15,342.26,452.25,301.62,678.29,599.64,418.86,914.50,490.72,493.08,396.94,653.56,489.89,358.16,325.29,312.48,306.42,368.27,298.17,587.29,497.04,845.52,359.98,422.28,284.90,354.42,496.38,351.04,413.84,519.57,552.18,353.97,726.80,413.13,377.45,317.04,260.26,601.85,457.55,603.79,420.70,408.63,415.28,352.38,531.92,511.50,493.21,686.92,241.23,378.85,339.07
82,2020-07-01,353.27,346.15,463.33,306.34,686.37,609.04,427.95,926.54,500.67,501.71,404.85,646.43,508.15,361.97,332.40,315.40,312.67,375.21,300.77,600.73,503.55,860.52,367.92,431.39,288.99,360.99,510.27,355.89,420.37,531.27,561.68,361.17,738.70,420.98,383.31,324.74,264.10,612.06,467.37,616.03,428.06,414.00,424.71,357.95,544.78,521.85,501.63,702.62,245.47,386.23,344.09
83,2020-10-01,360.35,350.13,479.53,313.35,700.35,621.55,439.86,946.23,508.68,514.58,413.12,658.40,531.56,366.47,340.43,320.29,319.12,382.70,304.58,622.46,512.94,877.90,375.37,439.02,294.45,368.34,526.28,363.05,429.53,547.61,573.55,368.91,755.28,431.43,387.98,332.46,268.94,628.74,475.67,634.69,436.43,423.89,434.36,365.90,563.74,538.42,510.84,721.18,248.92,394.06,351.01
84,2021-01-01,367.06,352.81,499.96,318.76,719.15,637.70,450.93,954.49,523.50,530.18,424.43,666.65,559.10,370.56,347.03,323.16,324.73,389.28,307.91,640.72,523.99,896.21,383.00,445.71,298.22,374.94,543.63,368.83,444.44,562.88,587.31,379.34,772.94,441.16,388.41,338.34,274.34,649.39,486.67,651.07,446.26,429.77,446.01,374.12,585.47,551.14,521.37,741.75,253.15,399.07,356.57


## Unemployment Rate


In [142]:
uri_filenames = "https://download.bls.gov/pub/time.series/la/la.txt"
r = requests.get(uri_filenames, allow_redirects=True)
open('filenames.txt', 'wb').write(r.content)
df = pd.read_fwf('filenames.txt', skiprows=104)
df.iloc[:,0] = df.iloc[:,0].str[:25]
filenames = [n.strip() for n in df.iloc[:52,0].tolist()]

In [143]:
dfs = []
uri = 'https://download.bls.gov/pub/time.series/la/'
for filename in filenames:
  r = requests.get(uri+filename, allow_redirects=True)
  open('temp.txt', 'wb').write(r.content)
  df = pd.read_fwf('temp.txt')
  df.drop(['footnote_codes'], axis=1,  inplace=True)
  index = df[~df.series_id.str.endswith('03')].iloc[0].name
  df = df.iloc[:index,:]
  dfs.append(df)

In [144]:
import functools
df_unemployment = functools.reduce(lambda left,right: left.append(right, ignore_index=True),dfs)
df_unemployment

,series_id,year,period,value
0,LASST010000000000003,1976,M01,6.6
1,LASST010000000000003,1976,M02,6.6
2,LASST010000000000003,1976,M03,6.6
3,LASST010000000000003,1976,M04,6.5
4,LASST010000000000003,1976,M05,6.4
...,...,...,...,...
27707,LASST560000000000003,2021,M06,5.4
27708,LASST560000000000003,2021,M07,5.2
27709,LASST560000000000003,2021,M08,4.9
27710,LASST560000000000003,2021,M09,4.5


## Interest Rate (World Bank)

In [145]:
url_interest_rate = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=INTDSRUSM193N&scale=left&cosd=1950-01-01&coed=2021-08-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2021-11-02&revision_date=2021-11-02&nd=1950-01-01"
r = requests.get(url_interest_rate, allow_redirects=True)
open('data-interest-rate.csv', 'wb').write(r.content)

13243

In [146]:
df_interest_rate = pd.read_csv('data-interest-rate.csv')
df_interest_rate

,DATE,INTDSRUSM193N
0,1950-01-01,1.50
1,1950-02-01,1.50
2,1950-03-01,1.50
3,1950-04-01,1.50
4,1950-05-01,1.50
...,...,...
855,2021-04-01,0.25
856,2021-05-01,0.25
857,2021-06-01,0.25
858,2021-07-01,0.25


## Population (US Census Bureau - cleaned by ISU)

In [147]:
url_population = "https://www.icip.iastate.edu/sites/default/files/uploads/tables/population/popest-annual.xls"
r = requests.get(url_population, allow_redirects=True)
open('data-population.xls', 'wb').write(r.content)

235008

In [148]:
url_population = 'https://www.icip.iastate.edu/sites/default/files/uploads/tables/population/popest-annual-historical.xls'
r = requests.get(url_population, allow_redirects=True)
open('data-population2009.xls', 'wb').write(r.content)

330240

In [149]:
df_population = pd.read_excel('data-population.xls', sheet_name="States", skiprows=6, skipfooter=8)
df_population.head(10)

,Fips,Area,Estimates Base (4/1/2010),2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0.0,United States,308758105.0,309327143.0,311583481.0,313877662.0,316059947.0,318386329.0,320738994.0,323071755.0,325122128.0,326838199.0,328329953.0
1,NaN,Northeast,55318414.0,55380764.0,55608318.0,55782661.0,55912775.0,56021339.0,56052790.0,56063777.0,56083383.0,56084543.0,56002934.0
2,NaN,Midwest,66929737.0,66975328.0,67164092.0,67348275.0,67576524.0,67765576.0,67885682.0,68018175.0,68160342.0,68263019.0,68340091.0
3,NaN,South,114563042.0,114869421.0,116019483.0,117264196.0,118397213.0,119666248.0,121049223.0,122419547.0,123611036.0,124649156.0,125686544.0
4,NaN,West,71946912.0,72101630.0,72791588.0,73482530.0,74173435.0,74933166.0,75751299.0,76570256.0,77267367.0,77841481.0,78300384.0
5,1.0,Alabama,4780118.0,4785514.0,4799642.0,4816632.0,4831586.0,4843737.0,4854803.0,4866824.0,4877989.0,4891628.0,4907965.0
6,2.0,Alaska,710246.0,713982.0,722349.0,730810.0,737626.0,737075.0,738430.0,742575.0,740983.0,736624.0,733603.0
7,4.0,Arizona,6392292.0,6407342.0,6473416.0,6556344.0,6634690.0,6732873.0,6832810.0,6944767.0,7048088.0,7164228.0,7291843.0
8,5.0,Arkansas,2916029.0,2921998.0,2941038.0,2952876.0,2960459.0,2968759.0,2979732.0,2991815.0,3003855.0,3012161.0,3020985.0
9,6.0,California,37254522.0,37319550.0,37636311.0,37944551.0,38253768.0,38586706.0,38904296.0,39149186.0,39337785.0,39437463.0,39437610.0


In [150]:
df_population = df_population[(df_population['Fips']>0) & (df_population['Fips']<57)]
df_population.reset_index(drop=True,inplace=True)
df_population.head(10)

,Fips,Area,Estimates Base (4/1/2010),2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1.0,Alabama,4780118.0,4785514.0,4799642.0,4816632.0,4831586.0,4843737.0,4854803.0,4866824.0,4877989.0,4891628.0,4907965.0
1,2.0,Alaska,710246.0,713982.0,722349.0,730810.0,737626.0,737075.0,738430.0,742575.0,740983.0,736624.0,733603.0
2,4.0,Arizona,6392292.0,6407342.0,6473416.0,6556344.0,6634690.0,6732873.0,6832810.0,6944767.0,7048088.0,7164228.0,7291843.0
3,5.0,Arkansas,2916029.0,2921998.0,2941038.0,2952876.0,2960459.0,2968759.0,2979732.0,2991815.0,3003855.0,3012161.0,3020985.0
4,6.0,California,37254522.0,37319550.0,37636311.0,37944551.0,38253768.0,38586706.0,38904296.0,39149186.0,39337785.0,39437463.0,39437610.0
5,8.0,Colorado,5029319.0,5047539.0,5121900.0,5193660.0,5270774.0,5352637.0,5454328.0,5543844.0,5617421.0,5697155.0,5758486.0
6,9.0,Connecticut,3574151.0,3579173.0,3588632.0,3595211.0,3595792.0,3595697.0,3588561.0,3579830.0,3575324.0,3574561.0,3566022.0
7,10.0,Delaware,897947.0,899647.0,907590.0,915518.0,924062.0,933131.0,942065.0,949989.0,957942.0,966985.0,976668.0
8,11.0,District of Columbia,601767.0,605282.0,620290.0,635737.0,651559.0,663603.0,677014.0,687576.0,697079.0,704147.0,708253.0
9,12.0,Florida,18804589.0,18846143.0,19055607.0,19302016.0,19551678.0,19853880.0,20219111.0,20627237.0,20977089.0,21254926.0,21492056.0


In [151]:
df_population2009 = pd.read_excel('data-population2009.xls', sheet_name="States", skiprows=4, skipfooter=4)
df_population2009.tail(10)

,Fips,Area Name,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
50,55,Wisconsin,4378000,4425944,4459968,4498433,4518461,4538173,4569616,4584688,4613299,4631944,4665911,4712045,4726345,4728862,4721438,4735571,4747765,4755625,4777923,4822391,4856568,4904562,4964343,5025398,5084889,5133678,5184836,5229986,5266213,5297672,5332666,5373999,5406835,5445162,5479203,5514026,5546166,5577655,5610775,5640996,5669264
51,56,Wyoming,329000,333795,340023,346863,353353,364548,380477,395446,411530,430897,451850,474185,491712,506400,510344,504895,499696,495633,476966,465103,458373,453690,459260,466251,473081,480283,485160,488167,489451,490787,491780,494300,494657,500017,503453,509106,514157,522667,534876,546043,559851
52,91,New England,11735000,11878088,11996218,12087834,12147736,12156540,12176402,12207302,12257357,12302714,12345269,12371656,12435782,12467917,12544350,12641838,12740544,12833004,12951233,13084523,13181740,13229502,13247630,13271045,13334364,13396380,13472634,13554814,13642197,13733828,13837653,13949721,14041043,14122441,14182454,14206683,14216656,14245540,14279205,14340115,14403575
53,92,Mideast,42111000,42516918,42869509,42991563,42836999,42709177,42727721,42667403,42546619,42421484,42358101,42272235,42328543,42382098,42543638,42686518,42793973,42991120,43189888,43435353,43584907,43761704,44070941,44392469,44717161,44969670,45186033,45383860,45579719,45822025,46105834,46386027,46619373,46841190,47034773,47162342,47239239,47334435,47473718,47684886,47943872
54,93,Great Lakes,39904000,40319722,40622322,40824274,40946800,41036791,41105385,41186726,41353165,41510152,41610978,41693671,41647869,41492474,41366046,41393218,41417995,41455030,41589687,41720866,41872543,42091157,42495903,42902625,43274741,43589980,43923694,44239468,44493603,44727877,44969225,45216019,45401564,45550284,45707787,45844372,45949142,46071573,46187992,46275120,46355854
55,94,Plains,16202000,16349829,16474813,16562762,16628032,16671561,16742551,16863608,16949656,17028437,17097176,17207567,17264245,17291648,17325258,17382070,17401690,17392728,17427930,17533255,17595235,17697550,17843335,18026179,18209613,18380746,18549556,18705269,18851351,18988053,19130836,19275412,19374967,19468009,19569167,19687933,19802730,19956982,20105697,20248815,20392583
56,95,Southeast,43440000,43973515,45013167,46019010,46991755,47954591,48787678,49513969,50311832,51113001,51976687,52881196,53627371,54248696,54856066,55514620,56198797,56861115,57536401,58120493,58733381,59516122,60500666,61508138,62530549,63574211,64601937,65611342,66655300,67626581,68569381,69497232,70317959,71151666,71961591,73019229,74148386,75065947,76122849,77057840,77814936
57,96,Southwest,16328000,16621375,17076950,17502870,17942658,18354229,18789070,19269802,19710280,20180482,20776513,21426140,21985332,22791219,23405364,23776476,24165954,24584827,24747513,24859623,25083118,25411251,25917326,26491437,27115790,27772489,28419852,29019737,29625063,30239551,30827272,31380654,31891889,32430969,32923761,33475468,34098071,34944948,35624083,36269039,36899289
58,97,Rocky Mountain,4943000,5037873,5194044,5368407,5527390,5649835,5781745,5915555,6078758,6256862,6438928,6592318,6742612,6903979,7034816,7108977,7167704,7199874,7205624,7202616,7234109,7305119,7477155,7696444,7938748,8171343,8379865,8565452,8745913,8918147,9094461,9268926,9430982,9567277,9675332,9807510,9972107,10189958,10406301,10609537,10793888
59,98,Far West,26635000,27101402,27570486,27918162,28327835,28800911,29346033,29929494,30553208,31285112,31964927,32779936,33433990,34086401,34716476,35321190,36037077,36815133,37640660,38542275,39534189,40610409,41427985,42225887,42797622,43271002,43744822,44314342,45053779,45798042,46505506,47188420,47891178,48493357,49053068,49601761,50090268,50570529,51031362,51608614,52167532


In [152]:
df_population2009 = df_population2009[(df_population2009['Fips']>0) & (df_population2009['Fips']<57)]
df_population2009.reset_index(drop=True,inplace=True)
DateCol = list(df_population2009.filter(regex='19[0-9][0-9].*'))
df_population2009.drop(DateCol, axis=1, inplace=True)
df_population2009.rename(columns={'Area Name':'Area'}, inplace=True)
df_population2009.head(10)

,Fips,Area,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,1,Alabama,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938
1,2,Alaska,627963,633714,642337,648414,659286,666946,675302,680300,687455,698895
2,4,Arizona,5160586,5273477,5396255,5510364,5652404,5839077,6029141,6167681,6280362,6343154
3,5,Arkansas,2678588,2691571,2705927,2724816,2749686,2781097,2821761,2848650,2874554,2896843
4,6,California,33987977,34479458,34871843,35253159,35574576,35827943,36021202,36250311,36604337,36961229
5,8,Colorado,4326921,4425687,4490406,4528732,4575013,4631888,4720423,4803868,4889730,4972195
6,9,Connecticut,3411777,3432835,3458749,3484336,3496094,3506956,3517460,3527270,3545579,3561807
7,10,Delaware,786373,795699,806169,818003,830803,845150,859268,871749,883874,891730
8,11,District of Columbia,572046,574504,573158,568502,567754,567136,570681,574404,580236,592228
9,12,Florida,16047515,16356966,16689370,17004085,17415318,17842038,18166990,18367842,18527305,18652644


In [153]:
x= df_population2009.merge(df_population, on=['Fips', 'Area'], how='inner')
x.drop('Estimates Base (4/1/2010)', axis=1, inplace=True)
x.to_csv('data-population-merged.csv', index=False)
df_population.columns[1]

'Area'

## HPA (US Census Bureau - cleaned by ISU) - local

In [154]:
df_hpa = pd.read_csv('HPA_formatted_05-19.csv')
df_hpa

,region,sub_region,State,FIPS_State_code,year,month,total_number_hpa
0,North East,New England,Connecticut,9,2005,January,651
1,North East,New England,Maine,23,2005,January,400
2,North East,New England,Massachusetts,25,2005,January,1544
3,North East,New England,New Hampshire,33,2005,January,497
4,North East,New England,Rhode Island,44,2005,January,128
...,...,...,...,...,...,...,...
9175,West,Pacific,Alaska,2,2019,December,117
9176,West,Pacific,California,6,2019,December,11523
9177,West,Pacific,Hawaii,15,2019,December,342
9178,West,Pacific,Oregon,41,2019,December,1821


## Google Search Volume Index (pytrends) - local


In [155]:
!pip install git+https://github.com/GeneralMills/pytrends.git
import pandas as pd
from pytrends.request import TrendReq
import datetime
import time

pytrend = TrendReq()

# Desired Keyword list (up to 5 search terms in a kw_list)
kw_list = ["Real Estate Agent"]

dt = pd.date_range(start='01/01/2007', end='31/12/2020', freq='M')
months = [d.strftime('%Y-%m-%d') for d in dt]

keyWords = ["Apartment","Home", "Lending","Real Estate Bubble","Broker","Home Equity","Mortgage", "Real Estate Investment","Bubble", "Housing Bubble", "Real Estate","Real Estate Listings","Construction","Housing Market","Real Estate Agent","Realtor","Flat","Investment","Real Estate Broker","Rent"]

def GVData(kw, months):
    pytrend = TrendReq(backoff_factor=0.1)
    kw_list = [kw]
    # get your initial data
    full_year = "{} {}".format(months[0],months[1])
    pytrend.build_payload(kw_list, cat=29, timeframe=full_year, geo='US') 
    #cat 29 corresponds to Realestate category see: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories , timeframe corresponds to 2004-present
    df_state = pytrend.interest_by_region() # Returns a pandas.dataframe of geoName and gt index [0, 100]
    # add YEAR and STATES column to data frame
    df_state["year"] = [months[0]]*51
    lastN = 0
    kStart = 1
    while(lastN < len(months)-2):
        try:
            for n,m in enumerate(months[kStart:-1]):        
                full_year = "{} {}".format(m,months[n+kStart+1])
                pytrend.build_payload(kw_list, cat=29, timeframe=full_year, geo='US') #cat 29 corresponds to Realestate category see: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories , timeframe corresponds to 2004-present
                df = pytrend.interest_by_region()
                # add the year and states column
                year = [m] * 51
                df["year"] = year
                #df["states"] = states 
                # append df to df_state
                df_state = df_state.append(df)
                lastN = n+ kStart
                print(m)
        except:
            print("Error", lastN)
        
        if(lastN < len(months)-2):
            kStart = lastN+1
            print("Sleeping for 60s")
            time.sleep(60)
            
    return df_state
	

for kw in keyWords[16:]:
    df = GVData(kw, months)
    fileName = "GSVI_" + '_'.join(kw.split()) + ".csv"
    df.to_csv(fileName)
    print(kw)

## Personal Income

In [188]:
df_personal_income = pd.read_csv('data-personal-income.csv', skiprows=1, skipfooter=4)
df_personal_income.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2004:Q1,2004:Q2,2004:Q3,2004:Q4,2005:Q1,2005:Q2,2005:Q3,2005:Q4,2006:Q1,2006:Q2,2006:Q3,2006:Q4,2007:Q1,2007:Q2,2007:Q3,2007:Q4,2008:Q1,2008:Q2,2008:Q3,2008:Q4,2009:Q1,2009:Q2,2009:Q3,2009:Q4,2010:Q1,2010:Q2,2010:Q3,2010:Q4,2011:Q1,2011:Q2,2011:Q3,2011:Q4,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,2014:Q3,2014:Q4,2015:Q1,2015:Q2,2015:Q3,2015:Q4,2016:Q1,2016:Q2,2016:Q3,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4,2019:Q1,2019:Q2,2019:Q3,2019:Q4,2020:Q1,2020:Q2,2020:Q3
0,"""01000""",Alabama,5,SQINC1,1,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,125134.6,127744.8,129787.1,132293.0,133147.8,135407.9,136796.6,139957.0,142775.3,144756.7,145895.9,147275.5,150248.4,152018.0,152837.4,154204.2,155057.1,161916.5,157016.7,155477.0,153254.1,156019.3,155440.1,155867.7,157933.1,160582.1,163381.2,164169.8,166803.6,167219.1,168544.8,169203.6,171357.6,173000.4,171345.9,175512.4,173553.3,174098.6,175331.4,174678.3,176504.1,179454.2,181869.0,184003.4,187042.8,188248.7,189748.3,189806.0,190785.4,191772.0,192487.5,194082.8,197343.1,199014.2,200457.5,203184.2,206139.7,207809.1,209457.0,211604.9,214330.6,215693.3,217250.7,218521.5,221867.9,241811.9,231119.5
1,"""02000""",Alaska *,8,SQINC1,1,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,23857.8,24100.2,24459.0,24936.4,25213.7,25650.7,26185.6,26777.6,27236.7,27647.9,27768.7,28253.3,29318.4,29868.5,29949.1,30219.4,32026.5,33079.0,32965.7,33229.1,32306.1,32919.4,33001.8,33358.5,34492.2,35060.8,35608.4,36013.9,36831.4,37576.7,38148.9,38754.3,38870.3,39201.0,39041.1,39564.6,38590.5,38606.2,39036.5,39204.7,40396.9,40986.3,41322.8,41735.5,42417.9,42548.1,42547.4,42510.0,41703.9,41657.6,41702.2,41918.8,41979.7,42178.3,42622.1,43037.4,43698.9,44029.6,44720.4,45029.4,45567.9,45859.5,46045.1,46307.9,46273.7,49558.6,48638.7
2,"""04000""",Arizona,6,SQINC1,1,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,163796.4,167634.7,170942.3,176181.5,180104.5,185768.7,190964.9,195707.9,204036.0,207287.3,210883.7,214300.8,217982.8,220343.8,222067.5,222549.5,222068.7,228910.1,223024.7,219379.8,211482.0,212575.3,210974.0,212869.9,211253.0,214400.8,216873.8,219566.9,224054.3,225241.5,227092.3,229073.1,233118.9,236159.8,235021.9,243083.8,238963.4,242095.2,244070.2,246295.4,252240.2,255552.6,259101.7,262747.3,267209.8,270421.1,271914.1,275487.0,277792.1,281898.3,285463.7,287642.8,293718.8,297154.0,302620.3,306536.6,313429.0,316844.0,321673.7,324958.7,330056.7,333403.4,336776.0,340736.3,345363.2,377958.3,369563.3
3,"""05000""",Arkansas,5,SQINC1,1,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,72186.1,73362.5,74393.2,76190.0,76212.1,77445.9,78111.2,80232.2,81597.5,82826.6,83449.5,84399.4,86176.3,87860.1,88502.7,91494.2,90215.8,94258.4,91743.5,91030.8,90558.6,90335.0,89823.8,90266.7,90743.7,92920.8,94583.9,94913.4,98258.7,98517.8,100063.1,101446.2,105118.0,106906.4,105879.3,110035.1,106680.0,107096.0,107678.9,107507.8,110690.4,113034.4,114276.5,116180.6,115946.4,117041.9,118509.8,119189.2,119554.2,120494.2,120937.8,122006.7,123768.0,124515.4,125063.1,126756.3,128233.0,129662.6,131105.1,132589.4,133285.4,134222.3,135222.7,136000.3,137467.4,151496.2,143133.3
4,"""06000""",California,8,SQINC1,1,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,1296704.1,1320981.0,1335921.0,1364009.9,1370109.8,1393535.6,1416893.4,1448603.1,1497168.9,1504015.5,1522251.7,1540793.8,1565699.3,1580476.3,1582387.7,1597729.9,1602984.4,1622798.6,1614538.7,1585997.8,1548522.0,1556315.3,1549974.7,1561241.3,1594061.6,1620381.4,1628231.2,1671156.5,1721992.6,1728856.2,1748051.5,1752859.8,1811394.3,1834222.3,1833201.9,1930771.5,1843565.7,1875122.9,1897293.5,1929524.3,1961846.5,2000617.2,2046033.8,2075644.9,2126849.0,2163796.5,2189368.4,2211707.0,2240100.3,2257599.7,2285459.1,231107

## State to Fips

In [189]:
url_state_fips = "https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv"
r = requests.get(url_state_fips, allow_redirects=True)
open('data-states-fips.csv', 'wb').write(r.content)
df_state_fips = pd.read_csv('data-states-fips.csv')
df_state_fips.rename(columns={' stusps': 'State', ' st':'Fips'}, inplace=True)
df_state_fips.drop(['stname'],axis=1,inplace=True)
df_state_fips = df_state_fips.astype({'Fips': 'int'})
df_state_fips['State'] = df_state_fips['State'].str.strip()
df_state_fips.head()

,Fips,State
0,1,AL
1,2,AK
2,4,AZ
3,5,AR
4,6,CA


# Data Cleaning

## House Price Index (`Year`, `Month`, `Fips`, `HPI`)

In [158]:
df = df_hpi.copy()
df = df.melt(id_vars=["DATE"], var_name="State", value_name="HPI")
df['State'] = df['State'].str[:2]
df = pd.merge(df, df_state_fips, how='left', on=['State'])
date = pd.DatetimeIndex(pd.to_datetime(df['DATE']))
df['Year'] = date.year
df['Month'] = date.month
df.drop(['DATE', 'State'], axis=1,  inplace=True)
df_hpi_cleaned = df
df_hpi_cleaned

,HPI,Fips,Year,Month
0,208.29,1,2000,1
1,210.06,1,2000,4
2,211.71,1,2000,7
3,214.38,1,2000,10
4,220.65,1,2001,1
...,...,...,...,...
4381,339.07,56,2020,4
4382,344.09,56,2020,7
4383,351.01,56,2020,10
4384,356.57,56,2021,1


## Unemployment Rate (`Year`, `Month`, `Unemployment Rate`)

In [177]:
df = df_unemployment.copy()
df['Fips'] = df['series_id'].str[5:7].astype('int')
df['Month'] = df['period'].str[1:3].astype('int')
df['Unemployment Rate'] = df['value'].str.strip()
df.drop(['series_id', 'period', 'value'], axis=1,  inplace=True)
df.rename(columns={'year': 'Year'}, inplace=True)
df_unemployment_cleaned = df

In [178]:
df_unemployment_cleaned.loc[df_unemployment_cleaned['Unemployment Rate'].str.contains('\t'), 'Unemployment Rate'] = df_unemployment_cleaned['Unemployment Rate'].str[:-3]
df_unemployment_cleaned.loc[df_unemployment_cleaned['Unemployment Rate'].str.contains('\.$'), 'Unemployment Rate'] = df_unemployment_cleaned['Unemployment Rate'].str[:-1]
df_unemployment_cleaned['Unemployment Rate'] = pd.to_numeric(df_unemployment_cleaned['Unemployment Rate'], errors='coerce')
df_unemployment_cleaned.to_csv("data-unemployment.csv", encoding='utf-8', index=False)
df_unemployment_cleaned

,Year,Fips,Month,Unemployment Rate
0,1976,1,1,6.6
1,1976,1,2,6.6
2,1976,1,3,6.6
3,1976,1,4,6.5
4,1976,1,5,6.4
...,...,...,...,...
27707,2021,56,6,5.4
27708,2021,56,7,5.2
27709,2021,56,8,4.9
27710,2021,56,9,4.5


## Interest Rate (`Year`, `Month`, `Interest Rate`)

In [161]:
df = df_interest_rate.copy()
date = pd.DatetimeIndex(pd.to_datetime(df['DATE']))
df['Year'] = date.year
df['Month'] = date.month
df.drop(['DATE'], axis=1,  inplace=True)
df.rename(columns={'INTDSRUSM193N': 'Interest Rate'}, inplace=True)
df = df.astype({'Year': 'int', 'Month': 'int'})
df_interest_rate_cleaned = df
df

,Interest Rate,Year,Month
0,1.50,1950,1
1,1.50,1950,2
2,1.50,1950,3
3,1.50,1950,4
4,1.50,1950,5
...,...,...,...
855,0.25,2021,4
856,0.25,2021,5
857,0.25,2021,6
858,0.25,2021,7


## Population (`Fip`, `Year`, `Population`)

In [162]:
df = x.melt(id_vars=["Fips", "Area"],\
                          var_name="Year",\
                          value_name="Population")
df.drop(['Area'], axis=1,  inplace=True)
df = df[df['Fips'].notna()]
df = df.astype({'Year': 'int', 'Fips': 'int'})
# df.to_csv('data-population.csv', index=False)
df_population = df
df

,Fips,Year,Population
0,1,2000,4452173.0
1,2,2000,627963.0
2,4,2000,5160586.0
3,5,2000,2678588.0
4,6,2000,33987977.0
...,...,...,...
1015,51,2019,8556642.0
1016,53,2019,7614024.0
1017,54,2019,1795263.0
1018,55,2019,5824581.0


## HPA (`Fips`, `Year`, `Month`, `HPA`) 

In [163]:
df = df_hpa.copy()
df.drop(['region', 'sub_region', 'State'], axis=1, inplace=True)
df.rename(columns={'FIPS_State_code': 'Fips', 'year':'Year', 'month':'Month', 'total_number_hpa':'HPA'}, inplace=True)
df['Month'] = pd.to_datetime(df['Month'], format="%B").dt.month
df = df.astype({'Year': 'int', 'Month': 'int', 'Fips': 'int'})
df_hpa_cleaned = df
# df_hpa_cleaned.to_csv('data-hpa.csv', index=False)
df

,Fips,Year,Month,HPA
0,9,2005,1,651
1,23,2005,1,400
2,25,2005,1,1544
3,33,2005,1,497
4,44,2005,1,128
...,...,...,...,...
9175,2,2019,12,117
9176,6,2019,12,11523
9177,15,2019,12,342
9178,41,2019,12,1821


## Personal Income

In [164]:
df = df_personal_income
df.drop(['TableName', 'LineCode', 'IndustryClassification', 'Description', 'Unit', 'Region', 'GeoName'], axis=1, inplace=True)
df.rename(columns={'GeoFIPS':'Fips'}, inplace=True)
df = df.melt(id_vars=["Fips"],\
             var_name="Year",\
             value_name="Personal Income")

df.loc[df['Year'].str.split(':', expand=True)[1].eq('Q1'), 'Month'] = 1
df.loc[df['Year'].str.split(':', expand=True)[1].eq('Q2'), 'Month'] = 4
df.loc[df['Year'].str.split(':', expand=True)[1].eq('Q3'), 'Month'] = 7
df.loc[df['Year'].str.split(':', expand=True)[1].eq('Q4'), 'Month'] = 11

df['Year'] = df['Year'].str.replace(":.*","").astype(int)
df['Fips'] = df['Fips'].str.slice(1,3).astype(int)
df['Month'] = df['Month'].astype(int)
# df.to_csv('data-personal-income-1.csv', index=False)
df_personal_income_cleaned = df
df

,Fips,Year,Personal Income,Month
0,1,2004,125134.6,1
1,2,2004,23857.8,1
2,4,2004,163796.4,1
3,5,2004,72186.1,1
4,6,2004,1296704.1,1
...,...,...,...,...
3948,94,2020,1208709.1,7
3949,95,2020,4470235.4,7
3950,96,2020,2288872.6,7
3951,97,2020,722972.6,7


# Merging

### Data Without HPI (Monthly)

In [165]:
# Interest Rates + Population
df_merge1 = pd.merge(df_population, df_interest_rate_cleaned,  how='inner', on = ['Year'])
# Interest Rates + Population + Unemployment
df_merge2 = pd.merge(df_merge1, df_unemployment_cleaned,  how='inner', on = ['Fips', 'Year', 'Month'])
# Interest Rates + Population + Unemployment + PI
df_merge3 = pd.merge(df_merge2, df_personal_income_cleaned,  how='inner', on = ['Fips', 'Year', 'Month'])
# Interest Rates + Population + Unemployment + PI + HPA
monthly_without_hpi = pd.merge(df_merge3, df_hpa_cleaned,  how='inner', on = ['Fips', 'Year', 'Month'])
monthly_without_hpi
# monthly_without_hpi.to_csv("monthly_without_hpi.csv", encoding='utf-8', index=False)

,Fips,Year,Population,Interest Rate,Month,Unemployment Rate,Personal Income,HPA
0,1,2005,4569805.0,3.25,1,5.0,133147.8,2986
1,1,2005,4569805.0,3.75,4,4.6,135407.9,2292
2,1,2005,4569805.0,4.25,7,4.3,136796.6,2663
3,1,2005,4569805.0,5.00,11,4.2,139957.0,2720
4,2,2005,666946.0,3.25,1,7.1,25213.7,90
...,...,...,...,...,...,...,...,...
3055,55,2019,5824581.0,2.25,11,3.3,312743.4,1949
3056,56,2019,580116.0,3.00,1,3.7,35852.7,103
3057,56,2019,580116.0,3.00,4,3.5,35821.9,207
3058,56,2019,580116.0,3.00,7,3.7,36107.5,173


### Data With HPI (Quarterly)

In [166]:
# Everything + HPI
quarterly_with_hpi = pd.merge(monthly_without_hpi, df_hpi_cleaned,  how='inner', on = ['Fips', 'Year', 'Month'])
quarterly_with_hpi
# quarterly_with_hpi.to_csv("quarterly_with_hpi.csv", encoding='utf-8', index=False)

,Fips,Year,Population,Interest Rate,Month,Unemployment Rate,Personal Income,HPA,HPI
0,1,2005,4569805.0,3.25,1,5.0,133147.8,2986,256.36
1,1,2005,4569805.0,3.75,4,4.6,135407.9,2292,262.41
2,1,2005,4569805.0,4.25,7,4.3,136796.6,2663,267.91
3,2,2005,666946.0,3.25,1,7.1,25213.7,90,238.92
4,2,2005,666946.0,3.75,4,6.9,25650.7,434,247.85
...,...,...,...,...,...,...,...,...,...
2290,55,2019,5824581.0,3.00,4,3.2,309377.0,1967,366.07
2291,55,2019,5824581.0,3.00,7,3.4,310060.3,1547,370.71
2292,56,2019,580116.0,3.00,1,3.7,35852.7,103,316.12
2293,56,2019,580116.0,3.00,4,3.5,35821.9,207,324.97


## Merging Formatted Data

In [ ]:
# df_population = pd.read_csv('data-population.csv')
# df_unemployment = pd.read_csv('data-unemployment.csv')
# df_interest_rate = pd.read_csv('data-interest-rate.csv')
# df_hpi = pd.read_csv('data-hpi.csv')
# df_hpa = pd.read_csv('data-hpa.csv')
# df_personal_income = pd.read_csv('data-personal-income-1.csv')

In [ ]:
# df_merge1 = pd.merge(df_population, df_unemployment,  how='inner', on = ['Fips','Year'])
# df_merge2 = pd.merge(df_merge1, df_interest_rate,  how='inner', on = ['Fips', 'Year', 'Month'])
# df_merge3 = pd.merge(df_merge2, df_hpi, how='inner', on = ['Fips', 'Year', 'Month'])
# df_merge4 = pd.merge(df_merge3, df_hpa, how='inner', on = ['Fips', 'Year', 'Month'])
# df_merge5 = pd.merge(df_merge4, df_personal_income, how='inner', on = ['Fips', 'Year', 'Month'])
# df_merge5